# Initializing Colab

Installing pytorch_xla

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

Downloading data

In [ ]:
!wget https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/prediction/v1.1/{sample.tar,aerial_map.tar,semantic_map.tar}
!wget https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/prediction/v1.1/{train.tar,validate.tar} 
!mkdir -p lyft/scenes/ lyft/aerial_map/ lyft/semantic_map/
!tar -xf sample.tar sample.zarr && mv sample.zarr lyft/scenes/ && rm sample.tar
!tar -xf aerial_map.tar -C lyft/ && rm aerial_map.tar
!tar -xf semantic_map.tar meta.json && mv meta.json lyft/
!tar -xf semantic_map.tar semantic_map.pb && mv semantic_map.pb lyft/semantic_map/ && rm semantic_map.tar

Cloning repository and installing dependencies

In [ ]:
!git clone https://github.com/VahidZee/ReasonAwareRasterizedTrajectoryPrediction.git
cd ReasonAwareRasterizedTrajectoryPrediction
pip install -r requirements.txt

# Importing dependencies

In [ ]:
import pytorch_lightning as pl
from l5kit.configs import load_config_data
from raster.lyft import LyftTrainerModule, LyftDataModule

## Initializing various parts

In [ ]:
config = load_config_data('~/config.yaml')
datamodule = LyftDataModule('~/lyft_data/', config)
training_procedure = LyftTrainerModule(config)
training_procedure.datamodule = datamodule

In [ ]:
checkpointing = pl.callbacks.ModelCheckpoint(verbose=True, monitor='loss/val')
# logger = pl.loggers.TensorBoardLogger()
trainer = pl.Trainer(log_save_interval=1, limit_train_batches=5,num_sanity_val_steps=0, limit_val_batches=5 , val_check_interval=0.5, checkpoint_callback=checkpointing, **config['train_params'].get('trainer', dict()))

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=lightning_logs

In [ ]:
trainer.fit(training_procedure)